# Converting STEP files to Unstructured OpenFOAM Mesh with the Pointwise Python API 

We have previously outlined how to generate an unstructured mesh with pointwise interactively, however, when processing many cases we might need to perform the process programatically. Pointwise provides a convenient python based API to allow us to automate the procedure, however, when initially creating the program for debugging purposes we will need to view the outputs using the GUI. To visualize the program we can directly connect our program to an active pointwise Glyph Server.

## Activating the Glyph Server

We start off by opening pointwise and opening the setting for our "Glyph Server":

![glyph server](./images/python_api/glyph_server.PNG)

We then set the "Listen Mode" to "Active" and take note of the "Network Port".

![glyph server](./images/python_api/glyph_server_port.PNG)

For the above case we see that our network port number is e 2807

## Connecting to Glyph Server in Python
First ensure that the pointwise [glyph client](https://pypi.org/project/pointwise-glyph-client/) is installed.
Once installed let us initialize a glyph client and poinwise api using the port number from our glyph server.

In [12]:
from pointwise import GlyphClient
# glyph client
# if we want to run the code without using the pointwise GUI we can set the port to 0
glf = GlyphClient(port=2807)
# pointwise api
pw = glf.get_glyphapi()

In [13]:
# we can reset the application with the following command
_ = pw.Application.reset()

## Setting the Default Connector Dimension Spacing and Grid Preference 
When generating new connectors or creating new domains we can assign a default spacing for generated points as well as the type of generated domain

In [14]:
    # set a variable to define the dimension spacing
    dimension_spacing = 0.2
    # set the connector calculation method and spacing
    pw.Connector.setCalculateDimensionMethod("Spacing")
    pw.Connector.setCalculateDimensionSpacing(dimension_spacing)
    # no set the grid preference to unstructured grids
    pw.Application.setGridPreference("Unstructured")

[]

## Importing the Surface Database and Generating Surface Domains
Some glyph object are transient which makes it convenient to use a Python context manager to automatically close the glyph. The following context manager creates an importer object and closes it once the database has been imported.

In [15]:
import pathlib as pt

input_filename = pt.Path("./example.STEP")

with pw.Application.begin("DatabaseImport") as databaseimport:
    databaseimport.initialize(
        "-strict", "-type", "Automatic", input_filename.absolute().as_posix()        
    )
    databaseimport.read()
    databaseimport.convert()

The Database which we want to use to create the domain is of type "Model". Let us get all the imported databases and search for the required model database

In [16]:
all_database_entities = pw.Database.getAll()
print("The total number of entities:", len(all_database_entities))
for entity in all_database_entities:
    if entity._type == "pw::Model":
        model_database = entity
print("The model database entity:", model_database)

The total number of entities: 142
The model database entity: ::pw::Model_15 (NONE-56)


As we now have the model database let us create domains along the database which will act as the walls of our mesh.

In [17]:

wall_domains = pw.DomainUnstructured.createOnDatabase(
    "-parametricConnectors", "Aligned", "-merge", 0, [model_database,]
)
# we have now generated the wall domains and stored them as a list
print(
    "[",
    (",\n").join([str(dom) for dom in wall_domains]),
    "]"
)

[ ::pw::DomainUnstructured_151 (dom-1),
::pw::DomainUnstructured_152 (dom-2),
::pw::DomainUnstructured_153 (dom-3),
::pw::DomainUnstructured_154 (dom-4),
::pw::DomainUnstructured_155 (dom-5),
::pw::DomainUnstructured_156 (dom-6),
::pw::DomainUnstructured_157 (dom-7),
::pw::DomainUnstructured_158 (dom-8),
::pw::DomainUnstructured_159 (dom-9),
::pw::DomainUnstructured_160 (dom-10),
::pw::DomainUnstructured_161 (dom-11),
::pw::DomainUnstructured_162 (dom-12) ]


# Creating the Domains for the Inlet and Outlets
When the domains are connected so too are connectors which outline our mesh. The inlet and outlet domains are not yet generated, thus, if we can isolate the inlet and outlet connectors can create the appropriate domains.

In [18]:
# Create the sets of connectors which describe the inlet and outlet domains.
# To do so we must know the appropriate names

inlet_connector_names    = ("con-1", "con-7")
outlet_1_connector_names = ("con-28", "con-31")
outlet_2_connector_names = ("con-35", "con-37")

inlet_outlet_connector_names = [inlet_connector_names, outlet_1_connector_names, outlet_2_connector_names]

# get the inlet and outlet connectort objects
inlet_outlet_connectors = [
    [pw.GridEntity.getByName(con_name) for con_name in con_pair] for con_pair in
    inlet_outlet_connector_names
]
# create the domains using the inlet and outlet connectors
inlet_outlet_domains = [
    pw.DomainUnstructured.createFromConnectors(connector_pair)[0] for
    connector_pair in inlet_outlet_connectors
]

print(
    "[",
    (",\n").join([str(dom) for dom in inlet_outlet_domains]),
    "]"
)

[ ::pw::DomainUnstructured_163 (dom-13),
::pw::DomainUnstructured_164 (dom-14),
::pw::DomainUnstructured_165 (dom-15) ]


### Converting the Inlet and Outlet Meshes to T-Rex Meshes and Refining
Once the meshes have been generated we can once again use a context manager to access an "UnstructuredSolver" to convert the mesh to a T-Rex mesh and refine.

In [19]:
trex_maximum_layers = 6
trex_growth_rate = 1.1
wall_spacing = dimension_spacing/8
# generate t-rex mesh for the inlet and outlet domains
with pw.Application.begin("UnstructuredSolver", inlet_outlet_domains) as unstruc_solver:
    # use the create factory method to generate a T-Rex condition and apply the
    # condition to the necessary domains and connectors
    trex_condition = pw.TRexCondition.create()
    trex_condition.setName("WALL")
    trex_condition.setConditionType("Wall")
    trex_condition.setValue(wall_spacing)
    # isolate the inlet and outlet domains and create a domain, connecter type set
    domain_connector_pairs = []
    for domain, connector_pair in zip(inlet_outlet_domains, inlet_outlet_connectors):
        for connector in connector_pair:
            domain_connector_pairs.append(
                [domain, connector, "Same"]
            )
    # apply the condition to all the connectors using the list of domain connector type sets
    trex_condition.apply(domain_connector_pairs)
    # create a collection of domains and set the unstructured solver attributes
    for domain in inlet_outlet_domains:
        domain.setUnstructuredSolverAttribute("TrexMaximumLayers", trex_maximum_layers)
        domain.setUnstructuredSolverAttribute("TrexGrowthRate", trex_growth_rate)
        domain.setUnstructuredSolverAttribute("TRexCellType", "TriangleQuad")
    # initialize and refine the domain
    unstruc_solver.run("Initialize")
    unstruc_solver.run("Refine")

# Combining the Mesh into a Block and Generating the Internal Mesh
As we only have the mesh for the boudary we need to generate the mesh of the internal field.

In [20]:
# create a list of all domains and create a block for meshing
all_domains = []
all_domains.extend(wall_domains)
all_domains.extend(inlet_outlet_domains)
# construct the block from all the domains
block = pw.BlockUnstructured.createFromDomains(all_domains)[0]

# set the boundary conditions of the block and then solve
with pw.Application.begin("UnstructuredSolver", block) as block_solver:
    # get/create the necessary boundary conditions
    wall_trex_condition  = pw.TRexCondition.getByName("WALL")
    match_trex_condition = pw.TRexCondition.create()
    match_trex_condition.setName("MATCH")

    # create the list required to be placed into the WALL bc
    block_wall_list = [ [block, dom, "Opposite"]  for dom in wall_domains ]
    # create the list required to be placed into the MATCH bc
    block_match_list = [ [block, dom, "Same"]  for dom in inlet_outlet_domains ]

    # finally apply the correct t-rex condition to domains
    wall_trex_condition.apply(block_wall_list)
    match_trex_condition.apply(block_match_list)
    # set the solver attributes
    block.setUnstructuredSolverAttribute("TRexMaximumLayers", trex_maximum_layers)
    block.setUnstructuredSolverAttribute("TRexGrowthRate", trex_growth_rate)
    # set allow even if incomplete
    block_solver.setStopWhenFullLayersNotMet("true")
    block_solver.setAllowIncomplete("true")
    # intiailize and refine the mesh
    block_solver.run("Initialize")
    block_solver.run("Refine")

# Create the Boundary Conditions, Resizing the Mesh and Saving the OpenFoam PolyMesh

After generating the mesh we can now assign the boundary conditions and convert it into OpenFoam PolyMesh format

In [21]:
output_directory = pt.Path("./example_polymesh/")
# set the CAE solver
pw.Application.setCAESolver("OpenFOAM")

# Create Boundary Conditions
wall_bc = pw.BoundaryCondition.create()
wall_bc.setName("WALL")
wall_bc.setPhysicalType("-usage", "CAE", "wall")

inlet_bc = pw.BoundaryCondition.create()
inlet_bc.setName("INLET")
inlet_bc.setPhysicalType("-usage", "CAE", "patch")

outlet_1_bc = pw.BoundaryCondition.create()
outlet_1_bc.setName("OUTLET-1")
outlet_1_bc.setPhysicalType("-usage", "CAE", "patch")

outlet_2_bc = pw.BoundaryCondition.create()
outlet_2_bc.setName("OUTLET-2")
outlet_2_bc.setPhysicalType("-usage", "CAE", "patch")

# ensure scale the model to ensure that the model size is in mm
with pw.Application.begin("Modify", [block]) as modifier:
    scaling  = Transform.scaling((0.001, 0.001, 0.001), anchor=(0,0,0))
    entities = modifier.getEntities()
    col      = pw.Collection()
    col.set(entities)
    pw.Entity.transform(scaling, col.list())


# assign the correct domains to the boundary conditions

wall_bc.apply([[block, dom] for dom in wall_domains])

inlet_bc.apply([block, inlet_outlet_domains[0]])
outlet_1_bc.apply([block, inlet_outlet_domains[1]])
outlet_2_bc.apply([block, inlet_outlet_domains[2]])

with pw.Application.begin("CaeExport") as cae_exporter:
    cae_exporter.addAllEntities()
    cae_exporter.initialize(
        "-strict", "-type", "CAE", output_directory.absolute().as_posix()
    )
    cae_exporter.verify()
    cae_exporter.write()